##  Aggregations


### Step 1: Initialize PySpark Session


In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lit , avg, coalesce , struct,array , explode, create_map,approx_count_distinct,sumDistinct, sum, mean, cube
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

from pyspark.sql.window import Window

from pyspark.sql import functions as F

# Create a Spark session
spark = SparkSession.builder.appName("day4").getOrCreate()


ImportError: cannot import name 'cube' from 'pyspark.sql.functions' (/home/rojesh/.local/lib/python3.10/site-packages/pyspark/sql/functions.py)

In [17]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "./US_Crime_Rates_1960_2014.csv"  # Replace with the actual path
US_df = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./US_Crime_Rates_1960_2014.csv"  # Replace with the actual path
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = "./titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)


In [18]:
US_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Violent: integer (nullable = true)
 |-- Property: integer (nullable = true)
 |-- Murder: integer (nullable = true)
 |-- Forcible_Rape: integer (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated_assault: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny_Theft: integer (nullable = true)
 |-- Vehicle_Theft: integer (nullable = true)



In [19]:
US_df.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

### count

Question: How many records are there in the US_Crime_Rates_1960_2014_df DataFrame?

In [20]:
count_us = US_df.count()

print(f"Total number of records : ", count_us)

Total number of records :  55


### countDistinct
Question: How many distinct years are present in the US_Crime_Rates_1960_2014_df DataFrame?
Answer:

In [21]:
#findinf the distinct values in the "Year" column:
distinct_years = US_df.select("Year").distinct().count()

#printing the result
print(f"Number of distinct years:" , distinct_years)

Number of distinct years: 55


### approx_count_distinct

Question: Estimate the approximate number of distinct values in the "Total" column of the US_Crime_Rates_1960_2014_df DataFrame.

In [22]:
#calculating the approximate count of distinct values in the given column


approx_distinct_total = US_df.select(approx_count_distinct("Total").alias("ApproxDistinctTotal")).first()[0]

#printing the outcome
print(f"Approximate distinct values in 'Total' column:", approx_distinct_total)


Approximate distinct values in 'Total' column: 55


###  first and last

Question: Find the first and last year in the US_Crime_Rates_1960_2014_df DataFrame.

In [23]:
#calculating the minimum year in the us_df and alias it as first_year
first_year = US_df.selectExpr("min(Year) as FirstYear").first()["FirstYear"]

#calculating the minimum year in the us_df and alias it as first_year
last_year = US_df.selectExpr("max(Year) as LastYear").first()["LastYear"]


#printing the outcome

print(f"First Year:", first_year)
print(f"Last Year:", last_year)

First Year: 1960
Last Year: 2014


### min and max

Question: Find the minimum and maximum population values in the US_Crime_Rates_1960_2014_df DataFrame.

In [24]:
#calculating the minimum population in the dataframe and alias it as "MinPopulation"
min_population = US_df.selectExpr("min(Population) as MinPopulation").first()["MinPopulation"]

#calculating the maximum population in the dataframe and alias it as "MaxPopulation"

max_population = US_df.selectExpr("max(Population) as MaxPopulation").first()["MaxPopulation"]

print(f"Minimum Population: {min_population}")
print(f"Maximum Population: {max_population}")

Minimum Population: 179323175
Maximum Population: 318857056


### sumDistinct

Question: Calculate the sum of distinct "Property" values for each year in the US_Crime_Rates_1960_2014_df DataFrame.

In [25]:
#calculating the sum of distinct values in the "Property" column for each year and aliasing the result as "SumDistinctProperty"
distinct_property_sum = US_df.groupBy("Year").agg(sumDistinct("Property").alias("SumDistinctProperty"))

#displaying the resulting dataframe
distinct_property_sum.show()



+----+-------------------+
|Year|SumDistinctProperty|
+----+-------------------+
|1990|           12655500|
|1975|           10252700|
|1977|            9955000|
|2003|           10442862|
|2007|            9843481|
|1974|            9278700|
|2006|            9983568|
|1978|           10123400|
|1961|            3198600|
|2013|            8650761|
|1988|           12356900|
|1997|           11558175|
|1994|           12131900|
|1968|            6125200|
|2014|            8277829|
|1973|            7842200|
|1979|           11041500|
|1971|            7771700|
|1966|            4793300|
|2004|           10319386|
+----+-------------------+
only showing top 20 rows



### avg

Question: Calculate the average "Murder" rate for the entire dataset in the US_Crime_Rates_1960_2014_df DataFrame.
Answer:

In [26]:
average_murder_rate = US_df.select(avg("Murder")).first()[0]

print(f"Average murder rate: {average_murder_rate}")

Average murder rate: 17317.236363636363


### Aggregating to Complex Types

Question: Calculate the total sum of "Violent" and "Property" crimes for each year in the US_Crime_Rates_1960_2014_df DataFrame. Store the results in a struct type column.

In [27]:
# Group the DataFrame 'US_df' by the "Year" column and calculate the sum of "Violent" crimes for each yearaliasing the result as "TotalViolentCrimes"

violent_crimes = US_df.groupBy("Year").agg(sum(col("Violent")).alias("TotalViolentCrimes"))
property_crimes = US_df.groupBy("Year").agg(sum(col("Property")).alias("TotalPropertyCrimes"))

#joining the two DataFrames on the "Year" column and create a struct column
crime_totals = violent_crimes.join(property_crimes, "Year", "inner") \
    .select("Year", struct(col("TotalViolentCrimes"), col("TotalPropertyCrimes")).alias("CrimeSums")) \
    .orderBy("Year")  # Sort by Year in ascending order

# Show the resulting DataFrame
crime_totals.show(truncate=False)

+----+-------------------+
|Year|CrimeSums          |
+----+-------------------+
|1960|{288460, 3095700}  |
|1961|{289390, 3198600}  |
|1962|{301510, 3450700}  |
|1963|{316970, 3792500}  |
|1964|{364220, 4200400}  |
|1965|{387390, 4352000}  |
|1966|{430180, 4793300}  |
|1967|{499930, 5403500}  |
|1968|{595010, 6125200}  |
|1969|{661870, 6749000}  |
|1970|{738820, 7359200}  |
|1971|{816500, 7771700}  |
|1972|{834900, 7413900}  |
|1973|{875910, 7842200}  |
|1974|{974720, 9278700}  |
|1975|{1039710, 10252700}|
|1976|{1004210, 10345500}|
|1977|{1029580, 9955000} |
|1978|{1085550, 10123400}|
|1979|{1208030, 11041500}|
+----+-------------------+
only showing top 20 rows



    ### Grouping

    Question: In the given US_Crime_Rates_1960_2014_df DataFrame, you are tasked with finding the average of all crimes combined for each year. Calculate the sum of all crime categories (Violent, Property, Murder, Forcible_Rape, Robbery, Aggravated_assault, Burglary, Larceny_Theft, Vehicle_Theft) for each year and then determine the average of these combined crime sums. Provide the result as the average of all crimes across the entire dataset.

In [52]:

# Combine multiple columns into one
combined_column = US_df.withColumn(
    "Combined_Total",
    col("Violent") + col("Property") + col("Murder") +
    col("Forcible_Rape") + col("Robbery") +
    col("Aggravated_assault") + col("Burglary") +
    col("Larceny_Theft") + col("Vehicle_Theft")
)
# Calculate the sum of the combined column
total_combined = combined_column.select(sum(col("Combined_Total")).alias("Total_Crime"))

# Calculate the count of rows in the DataFrame
row_count = US_df.count()

# Calculate the average by dividing the total sum by the row count
average_combined = total_combined.select((col("Total_Crime") / row_count).alias("AVG_Crime"))

# Show the average
average_combined.show()



Py4JJavaError: An error occurred while calling o298.count.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:76)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:484)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.C

### Window Functions

Question: Calculate the cumulative sum of "Property" values over the years using a window function in the US_Crime_Rates_1960_2014_df DataFrame.

In [29]:
#Creating a window specification
window_spec = Window.partitionBy("Year").orderBy("Year")

# Calculate the cumulative sum of "Property" values using the window function
US_df = US_df.withColumn(
    "CumulativePropertySum",
    sum("Property").over(window_spec)
)

# Select only the columns you want in the final DataFrame
US_df = US_df.select("Year", "Population", "Total", "Violent", "Property", "Murder", "Forcible_Rape", "Robbery", "Aggravated_assault", "Burglary", "Larceny_Theft", "Vehicle_Theft","CumulativePropertySum")

# Show the DataFrame
US_df.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|CumulativePropertySum|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|              3095700|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|              3198600|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|              3450700|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 10

In [30]:
pivoted_df = US_df.groupBy("Year").pivot("Crime_Type").agg(
    sum("Total").alias("Total"),
    sum("Violent").alias("Violent"),
    sum("Property").alias("Property"),
    sum("Murder").alias("Murder"),
    sum("Forcible_Rape").alias("Forcible_Rape"),
    sum("Robbery").alias("Robbery"),
    sum("Aggravated_assault").alias("Aggravated_assault"),
    sum("Burglary").alias("Burglary"),
    sum("Larceny_Theft").alias("Larceny_Theft"),
    sum("Vehicle_Theft").alias("Vehicle_Theft")
)

# Show the pivoted DataFrame
pivoted_df.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Crime_Type` cannot be resolved. Did you mean one of the following? [`Forcible_Rape`, `Property`, `Robbery`, `Burglary`, `Murder`].;
'Project ['Crime_Type]
+- Project [Year#1415, Population#1416, Total#1417, Violent#1418, Property#1419, Murder#1420, Forcible_Rape#1421, Robbery#1422, Aggravated_assault#1423, Burglary#1424, Larceny_Theft#1425, Vehicle_Theft#1426, CumulativePropertySum#2703L]
   +- Project [Year#1415, Population#1416, Total#1417, Violent#1418, Property#1419, Murder#1420, Forcible_Rape#1421, Robbery#1422, Aggravated_assault#1423, Burglary#1424, Larceny_Theft#1425, Vehicle_Theft#1426, CumulativePropertySum#2703L]
      +- Project [Year#1415, Population#1416, Total#1417, Violent#1418, Property#1419, Murder#1420, Forcible_Rape#1421, Robbery#1422, Aggravated_assault#1423, Burglary#1424, Larceny_Theft#1425, Vehicle_Theft#1426, CumulativePropertySum#2703L, CumulativePropertySum#2703L]

23/09/03 11:05:54 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

In [ ]:
crime_totals.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Total_Violent: long (nullable = true)
 |-- Total_Property: long (nullable = true)
 |-- Total_Murder: long (nullable = true)
 |-- Total_Forcible_Rape: long (nullable = true)
 |-- Total_Robbery: long (nullable = true)
 |-- Total_Aggravated_assault: long (nullable = true)
 |-- Total_Burglary: long (nullable = true)
 |-- Total_Larceny_Theft: long (nullable = true)
 |-- Total_Vehicle_Theft: long (nullable = true)

